In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
PATH =  '/content/drive/MyDrive/AIConnect/NLP_classificaiton/data'

train = pd.read_csv(os.path.join(PATH, 'train.csv'), encoding='utf-8')
test = pd.read_csv(os.path.join(PATH, 'test.csv'), encoding='utf-8')

In [4]:
# 두 라벨의 가능한 모든 조합 만들기
combinations = np.array(np.meshgrid(train.bias.unique(), train.hate.unique())).T.reshape(-1,2)

print(combinations)

[['none' 'none']
 ['none' 'hate']
 ['others' 'none']
 ['others' 'hate']
 ['gender' 'none']
 ['gender' 'hate']]


In [5]:
# bias, hate 컬럼을 합친 것
bias_hate = list(np.array([train['bias'].values, train['hate'].values]).T)

print(bias_hate[:5])


[array(['none', 'none'], dtype=object), array(['none', 'hate'], dtype=object), array(['others', 'hate'], dtype=object), array(['none', 'none'], dtype=object), array(['none', 'none'], dtype=object)]


In [6]:
labels = []
for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

train['label'] = labels
train.head(5)

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0


In [7]:
!pip install soynlp emoji

     |████████████████████████████████| 416 kB 5.1 MB/s 
     |████████████████████████████████| 174 kB 56.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=67e2e1059e343128f77b722744d7375f58fbc931ec2b506c553b67be6c97ecef
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [8]:
# 위 설명대로 데이터 전처리를 위한 함수 선언
import re
import emoji
from soynlp.normalizer import repeat_normalize

def clean(self, x):
    emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [ ]:
# 나중에 해주자

# Modeling

In [9]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 
     |████████████████████████████████| 6.8 MB 56.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [31]:
import os
import random
from tqdm import tqdm
import transformers
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

In [11]:
print(transformers.__version__)

4.16.2


In [12]:
# seed 고정, gpu 고정
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(2022)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Load Tokenizer, Model
Hugging Face Hub에 존재하는 Pretrained Tokenizer와 Model 및 Model Config 불러오기

이 때, Classification은 num_labels가 2로 Default되어있기 때문에 Model Config의 Parameter를  6으로 변경

In [13]:
모델 = 'beomi/KcELECTRA-base' # 보통 tokenizer도 같은거 씀. 이거만 바꾸면 모델, tokenizer 변경 가능.

In [14]:
MODEL_NAME = 모델

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config = AutoConfig.from_pretrained(MODEL_NAME)
config.num_labels = 6 # 6 labels

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

print(model)
print(config)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# Tokenizing

In [24]:
train_dataset, eval_dataset = train_test_split(train, test_size=0.1, shuffle=True, stratify=train['label'])

tokenized_train = tokenizer(
    list(train_dataset['title']),  
    list(train_dataset['comment']), # comment 만 학습에 사용하도록 title 제거
    return_tensors="pt",
    max_length=137, # Max_Length = 138  
    padding=True,
    truncation=True,
    add_special_tokens=True
)

tokenized_eval = tokenizer(
    list(eval_dataset['title']),
    list(eval_dataset['comment']),
    return_tensors="pt",
    max_length=137,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print(tokenized_train['input_ids'][0])
print(tokenizer.decode(tokenized_train['input_ids'][0]))

tensor([    2,     6,  3174,  4237,  4430, 25048,  9387,  4383,  4768, 36095,
           16, 25046, 10243, 14202, 25047,   780,  1419, 15049,  2247,  8220,
        25049,    12, 11869,    13,     6,     3, 36095,  4220, 11153, 12771,
         4198,  8329,    18,    18,  8600,  8051,  8254,  4020,  9023,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
[CLS] " 차예련 “ 25kg 감량, ‘ 아줌마 됐다 ’ 는 말 듣기 싫었다 ” ( 인터뷰 ) " [SEP] 감량만 한게 아닌거같던데.. 방송보다 누구지 했는데 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [25]:
train_dataset.head()

,title,comment,bias,hate,label
4218,"""차예련 “25kg 감량, ‘아줌마 됐다’는 말 듣기 싫었다”(인터뷰)""",감량만 한게 아닌거같던데..방송보다 누구지 했는데,none,none,0
3213,[SC초점]'미용실판 윤식당?'…'세빌리아 이발사' 이민정 투입→차별화 부각 성공했...,출연자들 하나같이 맘에 안드네 ㅎㅎ광규 형님 예능 식상하다 재미없고아재 개그 ㅠ 이...,others,hate,3
7135,"""'알함브라' 현빈, 박신혜 떠났다…김의성은 아들 손에 사망 (종합)""",와...개폼만 잡다가 궁나는 발암구라궁전. 스토리 다소 밋밋해도 감정선 하나는 10...,others,hate,3
3417,"""치타♥남연우, 여성래퍼와 배우의 만남 """"열애 중""""(종합)""",ㅋㅋㅋ으이그... 진짜 정신병자들 많다,none,hate,1
1710,“보디슈트→비닐의상→노브라 공항패션”..화사 파격 행보 향해 쏟아지는 응원[종합],키가 넘 작음 박나래랑 똑같던데,none,none,0


In [26]:

class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [27]:
# 원래는 문자열을 숫자로 바꿔서 label list에 담는건데 고장날까봐 그냥 사용함.
# 제대로 할거면 함수 지우고 label column 만 train_label 변수에 담으면 됨.
def label_to_num(label): 
    label_dict = {0 : 0, 1 : 1, 2 : 2, 3 : 3, 4 : 4, 5 : 5}  
    num_label = []

    for v in label: 
        num_label.append(label_dict[v])
    
    return num_label


train_label = label_to_num(train_dataset['label'].values)
eval_label = label_to_num(eval_dataset['label'].values)

In [28]:
train_label[:10]

[0, 3, 3, 1, 0, 1, 1, 1, 3, 3]

In [29]:
train_dataset = BERTDataset(tokenized_train, train_label)
eval_dataset = BERTDataset(tokenized_eval, eval_label)

print(train_dataset.__len__())
print(train_dataset.__getitem__(7529))
print(tokenizer.decode(train_dataset.__getitem__(7529)['input_ids'])) # 메서드 호출하면 이렇게 생겼구나 ..

7530
{'input_ids': tensor([    2,     6,    61, 15085,    63, 15509,  9435, 10657, 29841,  4161,
           16,  9991,    12, 14164,  4129,    13,  1628,  4028, 25053,  3828,
         4215, 17190,  4107,     6,     3,  9126,  4021, 15735,  1997,  4027,
         5759,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 46.3 MB/s 
     |████████████████████████████████| 144 kB 43.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=29ae73ef95e7fb3ec936ac522dc537d2d16e2bd4e22af908a5d885453d5986cc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [53]:
import wandb
from transformers import TrainingArguments, Trainer

wandb.init(project="4week_NLP", entity="team-5")

eval/accuracy,▁▄▅▅▆▅▇▄▃▃▃▄▆▅▇▆██
eval/f1,▆▁▃▅█▇▆▆▇▆
eval/f1_macro,▁▂▃▇▇██▄
eval/loss,▁▁▂▂▃▄▅▂▃▄▄▆▆▇▇███
eval/runtime,█▁▂▁▂▂▂▁▁▁▂▂▁▂▂▁▂▂
eval/samples_per_second,▁█▇█▇▇▇███▇▇▇▇▇█▇▇
eval/steps_per_second,▁█▇█▇▇▇███▇▇▇▇▇█▇▇
train/epoch,▁▂▂▃▃▄▄▅▅▆▆▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/global_step,▁▂▂▃▃▄▄▅▅▆▆▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/learning_rate,█▆▃█▆▃▁
train/loss,█▄▂▂▂▁▁


In [38]:
def compute_metrics(pred):
  """ validation을 위한 metrics function """
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  probs = pred.predictions

  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds) # 리더보드 평가에는 포함되지 않습니다.
  f1 = f1_score(labels, preds, average='macro')
  return {
      'accuracy': acc,
      'f1' : f1,
  }

In [39]:
training_ars = TrainingArguments(

    # 항상바꿔주자. checkpoint 마다 모델이 해당 경로에 저장됨.
    output_dir='/content/drive/MyDrive/AIConnect/NLP_classificaiton/result/kcelectra03',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    save_total_limit=7, # 성능 상위 5개 모델만 저장.  이거 용량 꽤 커서 제한 해줘야댐.
    save_strategy = 'epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end = True, # parameter들 의미를 정확히 모름 알아보고 바꿔주면 성능 올라갈듯.
    metric_for_best_model= 'f1',
    greater_is_better= True,
    fp16 = True,

)

trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [41]:
trainer.train()
model.save_pretrained('/content/drive/MyDrive/AIConnect/NLP_classificaiton/result/kcelectra03/best_model')

***** Running training *****
  Num examples = 7530
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2360


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

# prediction 을 위해 저장된 모델 load

In [42]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Tokenizer_NAME = 모델
tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)
  # checkpoint 마다 위에서 미리 지정해둔 경로에 모델 저장되어있음. 
# MODEL_NAME = '/content/drive/MyDrive/AIConnect/NLP_classificaiton/result/kcelectra02/checkpoint-3000'
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)

print(tokenizer)

loading configuration file https://huggingface.co/beomi/KcELECTRA-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/61dd2bdbb7e56ff51fdc66b6f0d1973d2d806cd616d38a149f1bfd2753babc3c.ba488f0d9624511a98ed83af3e8f6b33fe20b502e2cfb16ee9858a6b6f521982
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "BertTokenizer",
  "tra

PreTrainedTokenizerFast(name_or_path='beomi/KcELECTRA-base', vocab_size=50135, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [43]:
# test data에 label column을 추가해주자. test.csv 는 위에서 train 이랑 같이 불러왔음.
test['label'] = 0
test.tail(5)

,ID,title,comment,label
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,0
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,0
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
510,510,"오창석♥' 이채은, 웨딩사진?...순백의 드레스 입고 '활짝'",입에 손가릭이 10개 있으니 징그럽다,0


In [44]:
test.tail(5)

,ID,title,comment,label
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,0
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,0
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
510,510,"오창석♥' 이채은, 웨딩사진?...순백의 드레스 입고 '활짝'",입에 손가릭이 10개 있으니 징그럽다,0


In [46]:
# test data 도 tokenize 해주자.
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['title']),
    list(test['comment']),
    return_tensors="pt",
    max_length=137,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

print(test_dataset.__len__())
print(test_dataset.__getitem__(510))
print(tokenizer.decode(test_dataset.__getitem__(6)['input_ids']))

511
{'input_ids': tensor([    2,  2571,  4473,  4424, 27048,    11, 42252,  4192,    16, 31895,
        12869,    33,    18,    18,    18,  2155,  4529,  4041, 29372, 12634,
           11, 31971,    11,     3, 11010,  2111,  4050,  5222,  4012,  8229,
         4010,  8842, 23704,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask'

In [47]:
#test dataloader 선언
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
output_pred = []
output_prob = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
            token_type_ids=data['token_type_ids'].to(device)
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

100%|██████████| 32/32 [00:01<00:00, 21.14it/s]

[0, 0, 1, 1, 3, 1, 1, 1, 5, 0, 1, 5, 0, 3, 0, 5, 1, 1, 0, 0, 5, 1, 3, 1, 0, 1, 5, 0, 0, 0, 3, 5, 1, 3, 0, 5, 1, 0, 1, 0, 5, 0, 3, 0, 0, 1, 3, 1, 0, 5, 2, 0, 3, 5, 0, 3, 5, 0, 0, 1, 3, 4, 0, 5, 1, 1, 1, 3, 0, 5, 0, 1, 0, 1, 5, 0, 5, 2, 1, 1, 1, 1, 3, 0, 1, 0, 1, 3, 3, 0, 0, 0, 1, 3, 0, 5, 1, 3, 1, 1, 0, 3, 0, 0, 5, 0, 0, 1, 1, 0, 0, 5, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 1, 1, 5, 5, 0, 0, 0, 1, 0, 5, 1, 0, 1, 0, 1, 1, 0, 3, 5, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 5, 5, 0, 3, 3, 0, 0, 0, 3, 0, 0, 3, 5, 0, 1, 5, 0, 5, 0, 0, 0, 5, 1, 5, 0, 5, 0, 0, 3, 5, 3, 3, 3, 0, 3, 0, 0, 1, 1, 3, 1, 0, 0, 5, 1, 3, 0, 0, 0, 4, 1, 5, 0, 0, 1, 1, 3, 0, 3, 0, 1, 0, 4, 5, 1, 1, 1, 1, 1, 0, 1, 1, 0, 3, 0, 1, 0, 5, 0, 0, 1, 5, 3, 0, 0, 0, 3, 5, 3, 4, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 5, 5, 1, 0, 3, 0, 0, 0, 5, 5, 1, 0, 1, 5, 1, 0, 1, 0, 3, 1, 0, 3, 0, 5, 5, 0, 0, 0, 1, 5, 0, 1, 1, 1, 3, 0, 3, 1, 3, 1, 0, 1, 5, 4, 1, 0, 1, 0, 0, 1, 1, 1, 0, 3, 5, 5, 5, 1, 0, 5, 3, 1, 0, 1, 1, 3, 0, 3, 1, 0, 4, 1, 5, 0, 1, 

In [48]:
# 0-5 사이의 라벨 값 별로 bias, hate로 디코딩 하기 위한 딕셔너리
bias_dict = {0: 'none', 1: 'none', 2: 'others', 3:'others', 4:'gender', 5:'gender'}
hate_dict = {0: 'none', 1: 'hate', 2: 'none', 3:'hate', 4:'none', 5:'hate'}

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred_bias = ['' for i in range(len(pred_answer))]
pred_hate = ['' for i in range(len(pred_answer))]

for idx, label in enumerate(pred_answer):
    pred_bias[idx]=(str(bias_dict[label]))
    pred_hate[idx]=(str(hate_dict[label]))
print('decode Completed!')

decode Completed!


In [49]:
print(pred_bias)

['none', 'none', 'none', 'none', 'others', 'none', 'none', 'none', 'gender', 'none', 'none', 'gender', 'none', 'others', 'none', 'gender', 'none', 'none', 'none', 'none', 'gender', 'none', 'others', 'none', 'none', 'none', 'gender', 'none', 'none', 'none', 'others', 'gender', 'none', 'others', 'none', 'gender', 'none', 'none', 'none', 'none', 'gender', 'none', 'others', 'none', 'none', 'none', 'others', 'none', 'none', 'gender', 'others', 'none', 'others', 'gender', 'none', 'others', 'gender', 'none', 'none', 'none', 'others', 'gender', 'none', 'gender', 'none', 'none', 'none', 'others', 'none', 'gender', 'none', 'none', 'none', 'none', 'gender', 'none', 'gender', 'others', 'none', 'none', 'none', 'none', 'others', 'none', 'none', 'none', 'none', 'others', 'others', 'none', 'none', 'none', 'none', 'others', 'none', 'gender', 'none', 'others', 'none', 'none', 'none', 'others', 'none', 'none', 'gender', 'none', 'none', 'none', 'none', 'none', 'none', 'gender', 'others', 'none', 'others',

In [50]:
# submission 파일 가져오기
submit = pd.read_csv('/content/drive/MyDrive/AIConnect/NLP_classificaiton/data/sample_submission.csv') 

# submission 에 prediction 값 넣기
submit['bias'] = pred_bias
submit['hate'] = pred_hate
submit.head(5)

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate


In [51]:
# 항상 바꿔주자. submit 저장경로.
submit.to_csv('/content/drive/MyDrive/AIConnect/NLP_classificaiton/kcelectra_submit02.csv', index=False)

In [52]:
submit.head(7) # 잘됐는지 확인

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
5,5,none,hate
6,6,none,hate


# 제출하러가자 ㄱㄱㄱㄱ